In [ ]:
from importlib import import_module
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Path to your src folder
SRC_DIR = Path("../src").resolve()
sys.path.append(str(SRC_DIR))

# Import your modules
benchmark_mod = import_module("benchmark")
mom12_0_mod = import_module("momentum_12_0")
mom12_1_mod = import_module("momentum_12_1")

# Extract the dataframes
benchmark_df = benchmark_mod.benchmark_df
mom12_0_df = mom12_0_mod.mom12_0_df
mom12_1_df = mom12_1_mod.mom12_1_df
prev_perf_12_0 = mom12_0_mod.prev_perf_12_0
prev_perf_12_1 = mom12_1_mod.prev_perf_12_1
vol_perf_12_0 = mom12_0_mod.vol_perf_12_0
vol_perf_12_1 = mom12_1_mod.vol_perf_12_1


In [ ]:
# Calculate Maximum Drawdown
def max_drawdown(df):
    cumulative = df["cumvalue"]
    running_max = cumulative.cummax()
    drawdown = cumulative / running_max - 1
    return drawdown.min()

# Calculate Compound Annual Growth Rate (CAGR) from cumulative value
def cagr_from_cumvalue(cumvalue):
    n_years = len(cumvalue) / 12
    return cumvalue.iloc[-1] ** (1 / n_years) - 1

# Calculate summary stats: mean monthly return, standard deviation, Sharpe ratio, max drawdown
def summary_stats(df):
    mean_monthly = df["return"].mean()
    std_monthly = df["return"].std()
    cagr = cagr_from_cumvalue(df["cumvalue"])
    vol_annual = std_monthly * np.sqrt(12)
    sharpe = cagr / vol_annual
    mdd = max_drawdown(df)
    return mean_monthly, std_monthly, sharpe, mdd


In [ ]:
# Function to summarize strategy performance by market regime
def regime_summary(df, regime_col, strategy_name):
    summary = []
    for regime in df[regime_col].unique():
        r = df.loc[df[regime_col] == regime, "Long_Return"].dropna()
        n_months = len(r)
        mean_monthly = r.mean()
        monthly_vol = r.std()
        total_ret = (1 + r).prod() - 1
        invested_years = n_months / 12
        invested_cagr = (1 + total_ret) ** (1 / invested_years) - 1 if invested_years > 0 else np.nan
        vol_annual = monthly_vol * np.sqrt(12)
        sharpe = invested_cagr / vol_annual if vol_annual > 0 else np.nan
        summary.append({
            "Strategy": strategy_name,
            regime_col: regime,
            "Months": n_months,
            "Mean_Monthly_Return": mean_monthly,
            "Sharpe_Ratio": sharpe,
            "Total_Compounded_Return": total_ret,
            "Invested_CAGR": invested_cagr
        })
    return pd.DataFrame(summary)

# Function to create summary table for previous month's performance
def table_prev_month(df, strategy_name):
    summary_df = regime_summary(df, "PrevMonth_Regime", strategy_name)
    summary_df = summary_df.rename(columns={
        "PrevMonth_Regime": "Return Regime",
        "Mean_Monthly_Return": "Mean Monthly Return (%)",
        "Invested_CAGR": "Regime-Conditional CAGR (%)",
        "Total_Compounded_Return": "Total Compounded Return (%)"
    })
    # Convert percentages and clean up regime names
    summary_df["Mean Monthly Return (%)"] *= 100
    summary_df["Regime-Conditional CAGR (%)"] *= 100
    summary_df["Total Compounded Return (%)"] *= 100
    summary_df["Return Regime"] = summary_df["Return Regime"].replace({
        "Above Average Previous Return Month": "Above Average",
        "Below Average Previous Return Month": "Below Average"
    })
    return summary_df




In [ ]:
# Table 1: Summary statistics for different strategies
table1 = pd.DataFrame({
    "12-1 Momentum": summary_stats(mom12_1_df),
    "12-0 Momentum": summary_stats(mom12_0_df),
    "Benchmark": summary_stats(benchmark_df)
}, index=["Mean Monthly Return (%)", "Std. Dev. (Monthly)", "Sharpe Ratio", "Max Drawdown"])

# Convert percentages and round the table
table1.loc[["Mean Monthly Return (%)", "Std. Dev. (Monthly)", "Max Drawdown"]] *= 100
table1 = table1.round(2)


# Display the table
display(table1)


In [ ]:
# Table 2: Performance based on previous month's return regime
table2_12_1 = table_prev_month(prev_perf_12_1, "12-1")
table2_12_0 = table_prev_month(prev_perf_12_0, "12-0")
table2 = pd.concat([table2_12_1, table2_12_0], ignore_index=True)
display(table2.round(2))

In [ ]:
# Function to summarize performance by volatility regime
def table_volatility(df, strategy_name):
    summary_df = regime_summary(df, "Vol_Regime", strategy_name)
    summary_df = summary_df.rename(columns={
        "Vol_Regime": "Volatility Regime",
        "Mean_Monthly_Return": "Mean Monthly Return (%)",
        "Invested_CAGR": "Regime-Conditional CAGR (%)",
        "Total_Compounded_Return": "Total Compounded Return (%)"
    })
    # Convert percentages and clean up regime names
    summary_df["Mean Monthly Return (%)"] *= 100
    summary_df["Regime-Conditional CAGR (%)"] *= 100
    summary_df["Total Compounded Return (%)"] *= 100
    summary_df["Volatility Regime"] = summary_df["Volatility Regime"].replace({
        "Above Average Volatility": "Above Average",
        "Below Average Volatility": "Below Average"
    })
    return summary_df

# Table 3: Performance based on volatility regime
table3_12_1 = table_volatility(vol_perf_12_1, "12-1")
table3_12_0 = table_volatility(vol_perf_12_0, "12-0")
table3 = pd.concat([table3_12_1, table3_12_0], ignore_index=True)
display(table3.round(2))


In [ ]:
# Create joint regime columns combining previous month's and volatility regimes
def summarize_joint(joint_df, strategy_name):
    summary = []
    for regime in joint_df["Joint_Regime"].unique():
        r = joint_df.loc[joint_df["Joint_Regime"] == regime, "Long_Return"].dropna()
        n_months = len(r)
        if n_months == 0:
            continue

        mean_monthly = r.mean() * 100
        std_monthly = r.std()
        total_return = ((1 + r).prod() - 1) * 100
        invested_years = n_months / 12
        invested_cagr = ((1 + total_return / 100) ** (1 / invested_years) - 1) * 100
        sharpe = invested_cagr / (std_monthly * np.sqrt(12) * 100) if std_monthly > 0 else np.nan
        win_rate = (r > 0).mean()

        summary.append({
            "Strategy": strategy_name,
            "Joint Regime": regime,
            "Months": n_months,
            "Win_Rate": win_rate,
            "Mean Monthly Return (%)": mean_monthly,
            "Regime-Conditional CAGR (%)": invested_cagr,
            "Sharpe": sharpe,
            "Total Compounded Return (%)": total_return
        })
    return pd.DataFrame(summary)

# Combine previous month's regime and volatility regime for joint analysis
joint_12_1 = prev_perf_12_1.copy()
joint_12_1["Vol_Regime"] = vol_perf_12_1["Vol_Regime"]
joint_12_1["Joint_Regime"] = joint_12_1["PrevMonth_Regime"] + " & " + joint_12_1["Vol_Regime"]

joint_12_0 = prev_perf_12_0.copy()
joint_12_0["Vol_Regime"] = vol_perf_12_0["Vol_Regime"]
joint_12_0["Joint_Regime"] = joint_12_0["PrevMonth_Regime"] + " & " + joint_12_0["Vol_Regime"]

# Joint regime categories map
joint_regime_map = {
    "Above Average Previous Return Month & Below Average Volatility": "Stable Trend",
    "Below Average Previous Return Month & Below Average Volatility": "Slow Breakdown",
    "Above Average Previous Return Month & Above Average Volatility": "Overheated",
    "Below Average Previous Return Month & Above Average Volatility": "Stress / Crash"
}

# Function to create the joint regime summary table
def table_joint(joint_df, strategy_name):
    df = summarize_joint(joint_df, strategy_name)
    df["Joint Regime"] = df["Joint Regime"].replace(joint_regime_map)
    
    # Ensure correct order of joint regimes
    joint_regime_order = ["Stable Trend", "Slow Breakdown", "Overheated", "Stress / Crash"]
    df["Joint Regime"] = pd.Categorical(df["Joint Regime"], categories=joint_regime_order, ordered=True)
    df = df.sort_values("Joint Regime").reset_index(drop=True)
    
    # Reorder columns
    df = df[[
        "Strategy",
        "Joint Regime",
        "Months",
        "Mean Monthly Return (%)",
        "Regime-Conditional CAGR (%)",
        "Sharpe",
        "Total Compounded Return (%)"
    ]]
    return df

# Create joint regime tables for both 12-1 and 12-0 strategies
table4_12_1 = table_joint(joint_12_1, "12-1")
table4_12_0 = table_joint(joint_12_0, "12-0")

# Combine the tables for both strategies
table4 = pd.concat([table4_12_1, table4_12_0], ignore_index=True)

# Display the final joint regime table
display(table4.round(2))
